In [96]:
import pandas as pd
import os

In [97]:
df = pd.DataFrame()
files = os.listdir("./patients_new")
for file in files:
    patient = pd.read_json(os.path.join("./patients_new/", file))
    df = pd.concat([df, patient], ignore_index=True)

In [98]:
df

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,...,hcpcsevents,labevents,microbiologyevents,pharmacy,poe,prescriptions,procedures_icd,services,transfers,icustays
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,...,[],"[{'labevent_id': 151, 'subject_id': 10000032, ...","[{'microevent_id': 15, 'subject_id': 10000032,...","[{'subject_id': 10000032, 'hadm_id': 22595853,...","[{'poe_id': '10000032-52', 'poe_seq': 52, 'sub...","[{'subject_id': 10000032, 'hadm_id': 22595853,...","[{'subject_id': 10000032, 'hadm_id': 22595853,...","[{'subject_id': 10000032, 'hadm_id': 22595853,...","[{'subject_id': 10000032, 'hadm_id': 22595853....",[]
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,EW EMER.,P09Q6Y,EMERGENCY ROOM,HOME,Medicaid,...,[],"[{'labevent_id': 332, 'subject_id': 10000032, ...","[{'microevent_id': 28, 'subject_id': 10000032,...","[{'subject_id': 10000032, 'hadm_id': 22841357,...","[{'poe_id': '10000032-69', 'poe_seq': 69, 'sub...","[{'subject_id': 10000032, 'hadm_id': 22841357,...","[{'subject_id': 10000032, 'hadm_id': 22841357,...","[{'subject_id': 10000032, 'hadm_id': 22841357,...","[{'subject_id': 10000032, 'hadm_id': 22841357....",[]
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaN,EW EMER.,P60CC5,EMERGENCY ROOM,HOSPICE,Medicaid,...,[],"[{'labevent_id': 584, 'subject_id': 10000032, ...","[{'microevent_id': 36, 'subject_id': 10000032,...","[{'subject_id': 10000032, 'hadm_id': 25742920,...","[{'poe_id': '10000032-217', 'poe_seq': 217, 's...","[{'subject_id': 10000032, 'hadm_id': 25742920,...","[{'subject_id': 10000032, 'hadm_id': 25742920,...","[{'subject_id': 10000032, 'hadm_id': 25742920,...","[{'subject_id': 10000032, 'hadm_id': 25742920....",[]
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,EW EMER.,P30KEH,EMERGENCY ROOM,HOME,Medicaid,...,[],"[{'labevent_id': 407, 'subject_id': 10000032, ...","[{'microevent_id': 32, 'subject_id': 10000032,...","[{'subject_id': 10000032, 'hadm_id': 29079034,...","[{'poe_id': '10000032-125', 'poe_seq': 125, 's...","[{'subject_id': 10000032, 'hadm_id': 29079034,...",[],"[{'subject_id': 10000032, 'hadm_id': 29079034,...","[{'subject_id': 10000032, 'hadm_id': 29079034....","[{'subject_id': 10000032, 'hadm_id': 29079034,..."
4,10001217,24597018,2157-11-18 22:56:00,2157-11-25 18:00:00,NaN,EW EMER.,P4645A,EMERGENCY ROOM,HOME HEALTH CARE,Other,...,[],"[{'labevent_id': 9011, 'subject_id': 10001217,...","[{'microevent_id': 402, 'subject_id': 10001217...","[{'subject_id': 10001217, 'hadm_id': 24597018,...","[{'poe_id': '10001217-110', 'poe_seq': 110, 's...","[{'subject_id': 10001217, 'hadm_id': 24597018,...","[{'subject_id': 10001217, 'hadm_id': 24597018,...","[{'subject_id': 10001217, 'hadm_id': 24597018,...","[{'subject_id': 10001217, 'hadm_id': 24597018....","[{'subject_id': 10001217, 'hadm_id': 24597018,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,10040025,21791856,2147-06-16 22:00:00,2147-06-22 16:15:00,NaN,OBSERVATION ADMIT,P7634X,EMERGENCY ROOM,HOME HEALTH CARE,Other,...,[],"[{'labevent_id': 472612, 'subject_id': 1004002...","[{'microevent_id': 14525, 'subject_id': 100400...","[{'subject_id': 10040025, 'hadm_id': 21791856,...","[{'poe_id': '10040025-451', 'poe_seq': 451, 's...","[{'subject_id': 10040025, 'hadm_id': 21791856,...",[],"[{'subject_id': 10040025, 'hadm_id': 21791856,...","[{'subject_id': 10040025, 'hadm_id': 21791856....",[]
271,10040025,22251969,2147-08-03 02:58:00,2147-08-06 16:50:00,NaN,OBSERVATION ADMIT,P85BWS,EMERGENCY ROOM,HOME,Other,...,[],"[{'labevent_id': 472861, 'subject_id': 1004002...",[],"[{'subject_id': 10040025, 'hadm_id': 22251969,...","[{'poe_id': '10040025-594', 'poe_seq': 594, 's...","[{'subject_id': 10040025, 'hadm_id': 22251969,...",[],"[{'subject_id': 10040025, 'hadm_id': 22251969,...","[{'subject_id': 10040025, 'had

In [101]:
df["admission_type"].value_counts()

EW EMER.                       104
OBSERVATION ADMIT               45
URGENT                          38
EU OBSERVATION                  30
SURGICAL SAME DAY ADMISSION     18
DIRECT EMER.                    15
ELECTIVE                        13
DIRECT OBSERVATION               7
AMBULATORY OBSERVATION           5
Name: admission_type, dtype: int64

In [102]:
def normalize_jsons(column, dropper, sorter):
    data = []
    for datum in column:
        json_df = pd.json_normalize(datum)
        data.append(json_df)
    column_df = pd.concat(data, ignore_index=True).drop_duplicates(subset=dropper).sort_values(by=sorter).reset_index(drop=True)
    return column_df

In [103]:
patients_df = normalize_jsons(df["patients"], ["subject_id"], ["anchor_year"])
patients_df

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10008454,F,26,2110,2011 - 2013,
1,10001725,F,46,2110,2011 - 2013,
2,10006053,M,52,2111,2014 - 2016,2111-11-15
3,10004422,M,78,2111,2011 - 2013,
4,10035631,M,63,2112,2011 - 2013,2116-03-12
...,...,...,...,...,...,...
95,10020786,F,86,2189,2014 - 2016,
96,10002930,F,48,2193,2011 - 2013,2201-12-24
97,10004235,M,47,2196,2014 - 2016,
98,10026255,M,66,2200,2011 - 2013,2201-07-13


In [104]:
patients_df["gender"].value_counts()

M    57
F    43
Name: gender, dtype: int64

In [105]:
omr_df = normalize_jsons(df["omr"], ['subject_id', 'chartdate', 'result_name', 'result_value'], ["subject_id", "result_name", "chartdate"])
omr_df

,subject_id,chartdate,seq_num,result_name,result_value
0,10000032,2180-05-07,1,BMI (kg/m2),18.0
1,10000032,2180-05-25,1,BMI (kg/m2),18.6
2,10000032,2180-06-01,1,BMI (kg/m2),18.7
3,10000032,2180-06-22,1,BMI (kg/m2),18.9
4,10000032,2180-06-27,1,BMI (kg/m2),19.2
...,...,...,...,...,...
2834,10040025,2147-11-09,1,Weight (Lbs),188.00
2835,10040025,2147-12-05,1,Weight (Lbs),200.00
2836,10040025,2147-12-29,1,Weight (Lbs),212
2837,10040025,2147-12-30,1,Weight (Lbs),187.61


In [106]:
emar_df = normalize_jsons(df["emar"], ["subject_id", "hadm_id", "emar_id"], ["subject_id", "charttime"])
emar_df

,subject_id,hadm_id,emar_id,emar_seq,poe_id,pharmacy_id,enter_provider_id,charttime,medication,event_txt,scheduletime,storetime,0
0,10000032,22595853.0,10000032-11,11,10000032-22,14779570.0,,2180-05-07 00:44:00,Sodium Chloride 0.9% Flush,Flushed,2180-05-07 00:44:00,2180-05-07 00:44:00,"[{'subject_id': 10000032, 'emar_id': '10000032..."
1,10000032,22595853.0,10000032-10,10,10000032-36,48770010.0,,2180-05-07 00:44:00,Potassium Chloride,Administered,2180-05-07 00:44:00,2180-05-07 00:44:00,"[{'subject_id': 10000032, 'emar_id': '10000032..."
2,10000032,22595853.0,10000032-13,13,10000032-28,42497745.0,,2180-05-07 05:00:00,Albuterol Inhaler,Administered,2180-05-07 06:29:00,2180-05-07 06:29:00,"[{'subject_id': 10000032, 'emar_id': '10000032..."
3,10000032,22595853.0,10000032-12,12,10000032-37,93463122.0,,2180-05-07 06:10:00,Ipratropium Bromide Neb,Administered,2180-05-07 06:00:00,2180-05-07 06:10:00,"[{'subject_id': 10000032, 'emar_id': '10000032..."
4,10000032,22595853.0,10000032-17,17,10000032-22,14779570.0,,2180-05-07 07:51:00,Sodium Chloride 0.9% Flush,Flushed,2180-05-07 07:51:00,2180-05-07 07:56:00,"[{'subject_id': 10000032, 'emar_id': '10000032..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34825,10040025,27996267.0,10040025-3072,3072,10040025-1977,57820348.0,,2148-02-04 20:00:00,OxyCODONE SR (OxyconTIN),Not Given,2148-02-04 20:00:00,2148-02-04 19:47:00,"[{'subject_id': 10040025, 'emar_id': '10040025..."
34826,10040025,27996267.0,10040025-3066,3066,10040025-1978,,,2148-02-04 20:00:00,Senna,Not Given,2148-02-04 20:00:00,2148-02-04 19:42:00,"[{'subject_id': 10040025, 'emar_id': '10040025..."
34827,10040025,27996267.0,10040025-3065,3065,10040025-1997,,,2148-02-04 20:00:00,Polyethylene Glycol,Not Given,2148-02-04 20:00:00,2148-02-04 19:41:00,"[{'subject_id': 10040025, 'emar_id': '10040025..."
34828,10040025,27996267.0,10040025-3076,3076,10040025-1999,,,2148-02-04 22:00:00,,Not Confirmed,2148-02-04 22:00:00,2148-02-04 19:52:00,"[{'subject_id': 10040025, 'emar_id': '10040025..."


In [107]:
emar_df["medication"].value_counts()

Sodium Chloride 0.9%  Flush             3677
Insulin                                 2810
Heparin                                 1775
                                        1527
Metoprolol Tartrate                      942
                                        ... 
OxyCODONE--Acetaminophen (5mg-325mg)       1
Fosphenytoin                               1
Clarithromycin                             1
Nitroglycerin Ointment  2%                 1
Cisatracurium Besylate                     1
Name: medication, Length: 460, dtype: int64

In [108]:
dia_df = normalize_jsons(df["diagnoses_icd"], ["subject_id", "hadm_id", "seq_num", "icd_code"], ["subject_id", "seq_num"])
dia_df

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10000032,22595853,1,5723,9
1,10000032,22841357,1,07071,9
2,10000032,25742920,1,07054,9
3,10000032,29079034,1,45829,9
4,10000032,22595853,2,78959,9
...,...,...,...,...,...
4501,10040025,27996267,35,R197,10
4502,10040025,27996267,36,R748,10
4503,10040025,27996267,37,K5903,10
4504,10040025,27996267,38,T402X5A,10


In [109]:
transfer_df = normalize_jsons(df["transfers"], ["subject_id", "hadm_id", "transfer_id"], ["subject_id", "intime"])
transfer_df

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime
0,10000032,22595853.0,33258284,ED,Emergency Department,2180-05-06 19:17:00,2180-05-06 23:30:00
1,10000032,22595853.0,35223874,admit,Transplant,2180-05-06 23:30:00,2180-05-07 17:21:27
2,10000032,22595853.0,36904543,discharge,None,2180-05-07 17:21:27,None
3,10000032,22841357.0,38112554,ED,Emergency Department,2180-06-26 15:54:00,2180-06-26 21:31:00
4,10000032,22841357.0,34703856,admit,Transplant,2180-06-26 21:31:00,2180-06-27 18:49:12
...,...,...,...,...,...,...,...
1131,10040025,27996267.0,35449441,admit,PACU,2148-01-23 12:18:49,2148-01-23 14:23:57
1132,10040025,27996267.0,36822441,transfer,Vascular,2148-01-23 14:23:57,2148-01-24 04:50:17
1133,10040025,27996267.0,36107367,transfer,Surgical Intensive Care Unit (SICU),2148-01-24 04:50:17,2148-01-30 17:45:09
1134,10040025,27996267.0,36762745,transfer,Vascular,2148-01-30 17:45:09,2148-02-04 20:51:55


In [110]:
transfer_df["careunit"].value_counts()

Emergency Department                                182
Medicine                                             77
Med/Surg                                             48
Neurology                                            46
Medicine/Cardiology                                  43
Transplant                                           39
Cardiac Surgery                                      39
Medical Intensive Care Unit (MICU)                   36
Discharge Lounge                                     36
Surgical Intensive Care Unit (SICU)                  33
Medical/Surgical Intensive Care Unit (MICU/SICU)     32
Cardiac Vascular Intensive Care Unit (CVICU)         31
Hematology/Oncology                                  31
Emergency Department Observation                     26
PACU                                                 25
Med/Surg/Trauma                                      25
Hematology/Oncology Intermediate                     23
Trauma SICU (TSICU)                             

In [111]:
data = []
for datum in df["transfers"]:
    json_df = pd.json_normalize(datum)
    data.append(json_df)
column_df = pd.concat(data, ignore_index=True)
column_df

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime
0,10000032,22595853.0,36904543,discharge,None,2180-05-07 17:21:27,None
1,10000032,22595853.0,35223874,admit,Transplant,2180-05-06 23:30:00,2180-05-07 17:21:27
2,10000032,22595853.0,33258284,ED,Emergency Department,2180-05-06 19:17:00,2180-05-06 23:30:00
3,10000032,22841357.0,34100253,discharge,None,2180-06-27 18:49:12,None
4,10000032,22841357.0,34703856,admit,Transplant,2180-06-26 21:31:00,2180-06-27 18:49:12
...,...,...,...,...,...,...,...
1131,10040025,27996267.0,36822441,transfer,Vascular,2148-01-23 14:23:57,2148-01-24 04:50:17
1132,10040025,27996267.0,36041505,ED,Emergency Department,2148-01-22 14:47:00,2148-01-23 12:18:49
1133,10040025,27996267.0,36107367,transfer,Surgical Intensive Care Unit (SICU),2148-01-24 04:50:17,2148-01-30 17:45:09
1134,10040025,27125816.0,35704905,discharge,None,2143-03-19 12:03:31,None


In [113]:
subject_df = df.drop_duplicates(subset=["subject_id"])[["subject_id", "deathtime", "insurance", "language", "marital_status", "race"]]
patient_df = pd.merge(subject_df, patients_df, on='subject_id', how='inner')
patient_df

,subject_id,deathtime,insurance,language,marital_status,race,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,NaN,Other,ENGLISH,WIDOWED,WHITE,F,52,2180,2014 - 2016,2180-09-09
1,10001217,NaN,Other,?,MARRIED,WHITE,F,55,2157,2011 - 2013,
2,10001725,NaN,Other,ENGLISH,MARRIED,WHITE,F,46,2110,2011 - 2013,
3,10002428,NaN,Medicare,ENGLISH,WIDOWED,WHITE,F,80,2155,2011 - 2013,
4,10002495,NaN,Medicare,ENGLISH,MARRIED,UNKNOWN,M,81,2141,2014 - 2016,
...,...,...,...,...,...,...,...,...,...,...,...
95,10038999,NaN,Medicare,ENGLISH,SINGLE,WHITE,M,45,2131,2014 - 2016,
96,10039708,NaN,Other,ENGLISH,SINGLE,BLACK/AFRICAN AMERICAN,F,46,2138,2014 - 2016,
97,10039831,NaN,Other,ENGLISH,MARRIED,UNABLE TO OBTAIN,F,57,2115,2011 - 2013,
98,10039997,NaN,Medicare,ENGLISH,WIDOWED,BLACK/AFRICAN AMERICAN,F,67,2134,2014 - 2016,


In [114]:
patient_df["race"].value_counts()

WHITE                             63
UNKNOWN                           12
BLACK/AFRICAN AMERICAN             9
UNABLE TO OBTAIN                   3
OTHER                              3
HISPANIC/LATINO - SALVADORAN       2
BLACK/CAPE VERDEAN                 1
PORTUGUESE                         1
HISPANIC/LATINO - PUERTO RICAN     1
WHITE - BRAZILIAN                  1
HISPANIC OR LATINO                 1
WHITE - OTHER EUROPEAN             1
PATIENT DECLINED TO ANSWER         1
HISPANIC/LATINO - CUBAN            1
Name: race, dtype: int64

In [119]:
temp = patient_df.groupby(["race", "gender", "insurance"]).size().reset_index(name='count').sort_values(by=["count"], ascending=False)
temp['sum_counts'] = temp.groupby(["race", "gender"])['count'].transform('sum')
temp['ratio'] = temp['count'] / temp['sum_counts'] * 100
temp.sort_values(by=["race", "gender"])

,race,gender,insurance,count,sum_counts,ratio
1,BLACK/AFRICAN AMERICAN,F,Medicare,5,8,62.500000
2,BLACK/AFRICAN AMERICAN,F,Other,2,8,25.000000
0,BLACK/AFRICAN AMERICAN,F,Medicaid,1,8,12.500000
3,BLACK/AFRICAN AMERICAN,M,Medicare,1,1,100.000000
4,BLACK/CAPE VERDEAN,M,Medicaid,1,1,100.000000
5,HISPANIC OR LATINO,M,Other,1,1,100.000000
6,HISPANIC/LATINO - CUBAN,F,Medicare,1,1,100.000000
7,HISPANIC/LATINO - PUERTO RICAN,M,Medicaid,1,1,100.000000
9,HISPANIC/LATINO - SALVADORAN,F,Other,1,2,50.000000
8,HISPANIC/LATINO - SALVADORAN,F,Medicaid,1,2,50.000000
